In [5]:
#newsapi.ai key: d9052625-a43b-4f63-b861-c1c8abe115bc
#newsAPI key: c3645cc0bc8044bb97ddf2951e63c9fd
#!pip install requests_html

NewsAPI HTTP Status Codes
200	Success. Everything worked as expected.
204	Information not available. Request succeeded, but the requested information is not available.
400	Bad request. The request was unacceptable, most likely due to invalid or missing parameter.
401	User's limit reached. The user reached the limit of the tokens in his account. The requests are rejected.
403	Forbidden. The user's IP or account is disabled, or if all the tokens have been used. Check the error message for more details.
500	Internal error. Internal error occured while processing the request.
503	Service unavailable. The service is currently not available.

In [1]:
import requests  #to query the API 
import re  #regular expressions
import pandas as pd   # for dataframes
import json

import sklearn
from sklearn.feature_extraction.text import CountVectorizer  
 
#for text vectorization
#print(sklearn.__version__)

#from requests_html import HTMLSession
#from requests_html import AsyncHTMLSession

In [2]:

url = ('https://newsapi.org/v2/everything?'
        'q=(transgender+OR+trans+OR+binary)AND(youth+OR+minor+OR+minors+OR+children+OR+child)&'
       #'from=2021-02-06&' 
       #'to=2020-03-07&'
       #'pageSize=100&'
       #'language=en&'
       'apiKey=c3645cc0bc8044bb97ddf2951e63c9fd'
)
print(url)

https://newsapi.org/v2/everything?q=(transgender+OR+trans+OR+binary)AND(youth+OR+minor+OR+minors+OR+children+OR+child)&apiKey=c3645cc0bc8044bb97ddf2951e63c9fd


In [3]:
r = requests.get(url)
print(r)

<Response [200]>


In [5]:
jsontxt = r.json()

#print(jsontxt, "\n")

In [14]:
## Create a new csv file to save the headlines
filename='NewsAPI_jsontxt.csv'

In [8]:
MyFILE=open(filename,"w")
### Place the column names in - write to the first row
WriteThis="Author,Title,Headline\n"
MyFILE.write(WriteThis)
MyFILE.close()

In [9]:
#erases file content if you need to overwrite during troubleshooting
#MyFILE=open(filename,"r+")
#MyFILE.truncate(0)

In [10]:
## Open the file for append
MyFILE=open(filename, "a")
for items in jsontxt['articles']:
    #print(items) #prints each article as a dictionary
    #print(items['content'], "\n\n\n")
    #source=items["source"]["id"]
    #print(source, "\n")
    
    author=items["author"]
    if author == None:
        author = "None"
    author=re.sub(r'[,.;@#?!&$\-\']+', ' ', author, flags=re.IGNORECASE)
    author=re.sub(' +', ' ', author, flags=re.IGNORECASE)
    author=re.sub(r'\"', ' ', author, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    author=re.sub(r'[^a-zA-Z]', " ", author, flags=re.VERBOSE)
    author=author.replace(',', '')
    author=' '.join(author.split())
    author=re.sub("\n|\r", "", author)

    
    Title=items["title"]
    Title=re.sub(r'[,.;@#?!&$\-\']+', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(' +', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(r'\"', ' ', Title, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    Title=re.sub(r'[^a-zA-Z]', " ", Title, flags=re.VERBOSE)
    Title=Title.replace(',', '')
    Title=' '.join(Title.split())
    Title=re.sub("\n|\r", "", Title)
    #print(Title, '\n')
    
    #url = items['url']
    #print(url)

    Headline=items["description"]
    if Headline == "[Removed]":
        Headline = "None"
    Headline=re.sub(r'[,.;@#?!&$\-\']+', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(' +', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'\"', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'[^a-zA-Z]', " ", Headline, flags=re.VERBOSE)
    ## Be sure there are no commas in the headlines or it will
    ## write poorly to a csv file....
    Headline=Headline.replace(',', '')
    Headline=' '.join(Headline.split())
    Headline=re.sub("\n|\r", "", Headline)
    
    ### AS AN OPTION - remove words of a given length............
    #Headline = ' '.join([wd for wd in Headline.split() if len(wd)>3])

    #print("Author: ", Author, "\n")
    #print("Title: ", Title, "\n")
    #print("Headline News Item: ", Headline, "\n\n")
    
    WriteThis=str(author)+","+str(Title) + "," + str(Headline) + "\n"
    
    MyFILE.write(WriteThis)
    
## CLOSE THE FILE
MyFILE.close()

In [23]:
## Read to DF
newsAPI_df=pd.read_csv(filename, on_bad_lines = 'warn')
#print(newsAPI_df.head())
# iterating the columns 
#for col in newsAPI_df.columns: 
#    print(col) 
    
#df = pd.DataFrame(newsAPI_df)
#df
newsAPI_df['Label'] = "TransYouth"

In [24]:
newsAPI_df.head()

,Author,Title,Headline,Label
0,Kiara Alfonseca Lilia Geho,Ohio House votes to override governor s veto o...,The Ohio House has voted to override Gov Mike ...,TransYouth
1,Kiara Alfonseca Lilia Geho,Ohio Senate overrides governor veto of trans c...,The Ohio Senate voted to override Gov Mike DeW...,TransYouth
2,By ANDREW DeMILLO and HANNAH SCHOENBAUM AP,Efforts to Restrict Transgender Health Care En...,Republican led state legislatures are consider...,TransYouth
3,AMY BETH HANSON AP,Utah Joins Other States in Regulating Bathroom...,Utah became the latest state to regulate bathr...,TransYouth
4,Nilay Patel,Why Sen Brian Schatz thinks child safety bills...,The Democratic senator from Hawaii explains wh...,TransYouth


In [16]:

## REMOVE any rows with NaN in them
print(len(newsAPI_df["Headline"]))

newsAPI_df.drop(newsAPI_df[newsAPI_df['Headline'] == "None"].index, inplace = True)
newsAPI_df.drop(newsAPI_df[newsAPI_df['Author'] == "None"].index, inplace = True)

print(len(newsAPI_df["Headline"]))
#print(df.loc[30,:])

### Tokenize and Vectorize the Headlines
## Create the list of headlines
#HeadlineLIST=[]
#for next1 in newsAPI_df["Headline"]:
#    HeadlineLIST.append(next1)

#print("The headline list is")
#print(HeadlineLIST, len(HeadlineLIST))

100
93


In [14]:
#Request 2: LGBTQ Mental Health

url2 = ('https://newsapi.org/v2/everything?'
        'q=(LGBTQ)AND(mental)AND(health)&'
       #'from=2021-02-06&' 
       #'to=2020-03-07&'
       #'pageSize=100&'
       #'language=en&'
       'apiKey=c3645cc0bc8044bb97ddf2951e63c9fd'
)
print(url2)

https://newsapi.org/v2/everything?q=(LGBTQ)AND(mental)AND(health)&apiKey=c3645cc0bc8044bb97ddf2951e63c9fd


In [15]:
r2 = requests.get(url2)
print(r2)

<Response [200]>


In [16]:
jsontxt2 = r2.json()
#print(jsontxt2)

In [17]:
## Create a new csv file to save the headlines
filename2='NewsAPI_jsontxt2.csv'

In [18]:
MyFILE=open(filename2,"w")

WriteThis="Author,Title,Headline\n"
MyFILE.write(WriteThis)
MyFILE.close()


In [80]:
#If you need to empty the file for troubleshooting
#MyFILE=open(filename2,"r+")
#MyFILE.truncate(0)

0

In [19]:
## Open the file for append
MyFILE=open(filename2, "a")
for items in jsontxt2['articles']:
    #print(items) #prints each article as a dictionary
    #print(items['content'], "\n\n\n")
    #source=items["source"]["id"]
    #print(source, "\n")
    
    author=items["author"]
    if author == None:
        author = "None"
    author=re.sub(r'[,.;@#?!&$\-\']+', ' ', author, flags=re.IGNORECASE)
    author=re.sub(' +', ' ', author, flags=re.IGNORECASE)
    author=re.sub(r'\"', ' ', author, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    author=re.sub(r'[^a-zA-Z]', " ", author, flags=re.VERBOSE)
    author=author.replace(',', '')
    author=' '.join(author.split())
    author=re.sub("\n|\r", "", author)

    
    Title=items["title"]
    Title=re.sub(r'[,.;@#?!&$\-\']+', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(' +', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(r'\"', ' ', Title, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    Title=re.sub(r'[^a-zA-Z]', " ", Title, flags=re.VERBOSE)
    Title=Title.replace(',', '')
    Title=' '.join(Title.split())
    Title=re.sub("\n|\r", "", Title)
    #print(Title, '\n')
    
    #url = items['url']
    #print(url)

    Headline=items["description"]
    if Headline == None:
        Headline = "None"
    Headline=re.sub(r'\n', ' ', Headline, flags=re.IGNORECASE)
    #print(Headline)
    Headline=re.sub(r'[,.;@#?!&$\-\']+', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(' +', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'\"', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'[^a-zA-Z]', " ", Headline, flags=re.VERBOSE)
    ## Be sure there are no commas in the headlines or it will
    ## write poorly to a csv file....
    Headline=Headline.replace(',', '')
    Headline=' '.join(Headline.split())
    Headline=re.sub("\n|\r", "", Headline)
    
    ### AS AN OPTION - remove words of a given length............
    #Headline = ' '.join([wd for wd in Headline.split() if len(wd)>3])

    #print("Author: ", Author, "\n")
    #print("Title: ", Title, "\n")
    #print("Headline News Item: ", Headline, "\n\n")
    
    WriteThis=str(author)+","+str(Title) + "," + str(Headline) + "\n"
    
    MyFILE.write(WriteThis)
    
## CLOSE THE FILE
MyFILE.close()

In [18]:
## Read to DF

## Read to DF
newsAPI_df2=pd.read_csv(filename2, on_bad_lines = 'warn')

## REMOVE any rows with NaN in them
print(len(newsAPI_df2["Headline"]))

newsAPI_df2.drop(newsAPI_df2[newsAPI_df2['Headline'] == "None"].index, inplace = True)
newsAPI_df2.drop(newsAPI_df2[newsAPI_df2['Author'] == "None"].index, inplace = True)

print(len(newsAPI_df2["Headline"]))
#print(df.loc[30,:])

### Tokenize and Vectorize the Headlines
## Create the list of headlines
#HeadlineLIST2=[]
#for next1 in newsAPI_df2["Headline"]:
#    HeadlineLIST2.append(next1)

#print("The headline list is")
#print(HeadlineLIST2, len(HeadlineLIST2))


100
95


In [25]:
newsAPI_df2['Label'] = "LGBTQ_MH"

In [21]:
#Request 3: Gender 14-24

url3 = ('https://newsapi.org/v2/everything?'
        'q=(gender)&'
       'from=2024-02-06&' 
       'to=2014-01-01&'
       #'pageSize=100&'
       #'language=en&'
       'apiKey=c3645cc0bc8044bb97ddf2951e63c9fd'
)
print(url3)

https://newsapi.org/v2/everything?q=(gender)&from=2024-02-06&to=2014-01-01&apiKey=c3645cc0bc8044bb97ddf2951e63c9fd


In [22]:
r3 = requests.get(url3)
print(r3)

<Response [200]>


In [23]:
jsontxt3 = r3.json()
#print(jsontxt3)

In [19]:
filename3='NewsAPI_jsontxt3.csv'
#If you need to empty the file for troubleshooting
#MyFILE=open(filename3,"r+")
#MyFILE.truncate(0)

In [26]:
MyFILE=open(filename3,"w")

WriteThis="Author,Title,Headline\n"
MyFILE.write(WriteThis)
MyFILE.close()

In [27]:
## Open the file for append
MyFILE=open(filename3, "a")
for items in jsontxt3['articles']:
    #print(items) #prints each article as a dictionary
    #print(items['content'], "\n\n\n")
    #source=items["source"]["id"]
    #print(source, "\n")
    
    author=items["author"]
    if author == None:
        author = "None"
    author=re.sub(r'[,.;@#?!&$\-\']+', ' ', author, flags=re.IGNORECASE)
    author=re.sub(' +', ' ', author, flags=re.IGNORECASE)
    author=re.sub(r'\"', ' ', author, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    author=re.sub(r'[^a-zA-Z]', " ", author, flags=re.VERBOSE)
    author=author.replace(',', '')
    author=' '.join(author.split())
    author=re.sub("\n|\r", "", author)

    
    Title=items["title"]
    Title=re.sub(r'[,.;@#?!&$\-\']+', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(' +', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(r'\"', ' ', Title, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    Title=re.sub(r'[^a-zA-Z]', " ", Title, flags=re.VERBOSE)
    Title=Title.replace(',', '')
    Title=' '.join(Title.split())
    Title=re.sub("\n|\r", "", Title)
    #print(Title, '\n')
    
    #url = items['url']
    #print(url)

    Headline=items["description"]
    if Headline == None:
        Headline = "None"
    Headline=re.sub(r'\n', ' ', Headline, flags=re.IGNORECASE)
    #print(Headline)
    Headline=re.sub(r'[,.;@#?!&$\-\']+', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(' +', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'\"', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'[^a-zA-Z]', " ", Headline, flags=re.VERBOSE)
    ## Be sure there are no commas in the headlines or it will
    ## write poorly to a csv file....
    Headline=Headline.replace(',', '')
    Headline=' '.join(Headline.split())
    Headline=re.sub("\n|\r", "", Headline)
    
    ### AS AN OPTION - remove words of a given length............
    #Headline = ' '.join([wd for wd in Headline.split() if len(wd)>3])

    #print("Author: ", Author, "\n")
    #print("Title: ", Title, "\n")
    #print("Headline News Item: ", Headline, "\n\n")
    
    WriteThis=str(author)+","+str(Title) + "," + str(Headline) + "\n"
    
    MyFILE.write(WriteThis)
    
## CLOSE THE FILE
MyFILE.close()

In [20]:
## Read to DF

## Read to DF
newsAPI_df3=pd.read_csv(filename3, on_bad_lines = 'warn')

## REMOVE any rows with NaN in them
print(len(newsAPI_df3["Headline"]))

newsAPI_df3.drop(newsAPI_df3[newsAPI_df3['Headline'] == "None"].index, inplace = True)
newsAPI_df3.drop(newsAPI_df3[newsAPI_df3['Author'] == "None"].index, inplace = True)

print(len(newsAPI_df3["Headline"]))
#print(df.loc[30,:])

### Tokenize and Vectorize the Headlines
## Create the list of headlines
#HeadlineLIST2=[]
#for next1 in newsAPI_df2["Headline"]:
#    HeadlineLIST2.append(next1)

#print("The headline list is")
#print(HeadlineLIST2, len(HeadlineLIST2))


100
95


In [26]:
newsAPI_df3['Label'] = "Gender_14-24"

In [8]:
#Request 4: Gender Policy 04-14 Will have to convert csvs to dataframes before merging but only need to do two calls


url4 = ('https://newsapi.org/v2/everything?'
        'q=(gender)AND(policy)&'
       'from=2024-02-06&' 
       'to=2014-01-01&'
       #'pageSize=100&'
       #'language=en&'
       'apiKey=c3645cc0bc8044bb97ddf2951e63c9fd'
)
print(url4)

https://newsapi.org/v2/everything?q=(gender)AND(policy)&from=2024-02-06&to=2014-01-01&apiKey=c3645cc0bc8044bb97ddf2951e63c9fd


In [9]:
r4 = requests.get(url4)
print(r4)

<Response [200]>


In [10]:
jsontxt4 = r4.json()
#print(jsontxt3)

In [21]:

filename4='NewsAPI_jsontxt4.csv'
#If you need to empty the file for troubleshooting
#MyFILE=open(filename3,"r+")
#MyFILE.truncate(0)

In [12]:
MyFILE=open(filename4,"w")

WriteThis="Author,Title,Headline\n"
MyFILE.write(WriteThis)
MyFILE.close()

In [13]:
## Open the file for append
MyFILE=open(filename4, "a")
for items in jsontxt4['articles']:
    #print(items) #prints each article as a dictionary
    #print(items['content'], "\n\n\n")
    #source=items["source"]["id"]
    #print(source, "\n")
    
    author=items["author"]
    if author == None:
        author = "None"
    author=re.sub(r'[,.;@#?!&$\-\']+', ' ', author, flags=re.IGNORECASE)
    author=re.sub(' +', ' ', author, flags=re.IGNORECASE)
    author=re.sub(r'\"', ' ', author, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    author=re.sub(r'[^a-zA-Z]', " ", author, flags=re.VERBOSE)
    author=author.replace(',', '')
    author=' '.join(author.split())
    author=re.sub("\n|\r", "", author)

    
    Title=items["title"]
    Title=re.sub(r'[,.;@#?!&$\-\']+', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(' +', ' ', Title, flags=re.IGNORECASE)
    Title=re.sub(r'\"', ' ', Title, flags=re.IGNORECASE)
    
    # and replace it with a single space
    ## NOTE: Using the "^" on the inside of the [] means
    ## we want to look for any chars NOT a-z or A-Z and replace
    ## them with blank. This removes chars that should not be there.
    Title=re.sub(r'[^a-zA-Z]', " ", Title, flags=re.VERBOSE)
    Title=Title.replace(',', '')
    Title=' '.join(Title.split())
    Title=re.sub("\n|\r", "", Title)
    #print(Title, '\n')
    
    #url = items['url']
    #print(url)

    Headline=items["description"]
    if Headline == None:
        Headline = "None"
    Headline=re.sub(r'\n', ' ', Headline, flags=re.IGNORECASE)
    #print(Headline)
    Headline=re.sub(r'[,.;@#?!&$\-\']+', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(' +', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'\"', ' ', Headline, flags=re.IGNORECASE)
    Headline=re.sub(r'[^a-zA-Z]', " ", Headline, flags=re.VERBOSE)
    ## Be sure there are no commas in the headlines or it will
    ## write poorly to a csv file....
    Headline=Headline.replace(',', '')
    Headline=' '.join(Headline.split())
    Headline=re.sub("\n|\r", "", Headline)
    
    ### AS AN OPTION - remove words of a given length............
    #Headline = ' '.join([wd for wd in Headline.split() if len(wd)>3])

    #print("Author: ", Author, "\n")
    #print("Title: ", Title, "\n")
    #print("Headline News Item: ", Headline, "\n\n")
    
    WriteThis=str(author)+","+str(Title) + "," + str(Headline) + "\n"
    
    MyFILE.write(WriteThis)
    
## CLOSE THE FILE
MyFILE.close()

In [22]:
## Read to DF

## Read to DF
newsAPI_df4=pd.read_csv(filename4, on_bad_lines = 'warn')

## REMOVE any rows with NaN in them
print(len(newsAPI_df4["Headline"]))

newsAPI_df4.drop(newsAPI_df4[newsAPI_df4['Headline'] == "None"].index, inplace = True)
newsAPI_df4.drop(newsAPI_df4[newsAPI_df4['Author'] == "None"].index, inplace = True)

print(len(newsAPI_df4["Headline"]))
#print(df.loc[30,:])

### Tokenize and Vectorize the Headlines
## Create the list of headlines
#HeadlineLIST2=[]
#for next1 in newsAPI_df2["Headline"]:
#    HeadlineLIST2.append(next1)

#print("The headline list is")
#print(HeadlineLIST2, len(HeadlineLIST2))


100
93


In [27]:
newsAPI_df4['Label'] = "Policy_14-24"

In [28]:
combined_df = pd.concat([newsAPI_df,newsAPI_df2, newsAPI_df3, newsAPI_df4], ignore_index=True)

combined_df.head()

,Author,Title,Headline,Label
0,Kiara Alfonseca Lilia Geho,Ohio House votes to override governor s veto o...,The Ohio House has voted to override Gov Mike ...,TransYouth
1,Kiara Alfonseca Lilia Geho,Ohio Senate overrides governor veto of trans c...,The Ohio Senate voted to override Gov Mike DeW...,TransYouth
2,By ANDREW DeMILLO and HANNAH SCHOENBAUM AP,Efforts to Restrict Transgender Health Care En...,Republican led state legislatures are consider...,TransYouth
3,AMY BETH HANSON AP,Utah Joins Other States in Regulating Bathroom...,Utah became the latest state to regulate bathr...,TransYouth
4,Nilay Patel,Why Sen Brian Schatz thinks child safety bills...,The Democratic senator from Hawaii explains wh...,TransYouth


In [30]:
combined_df.to_csv('Combined_DF_NewsAPI.csv')

In [32]:
len(combined_df)

383

/tmp/ipykernel_128/4292724039.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = df.append(df2)


,Author,Title,Headline
0,Kiara Alfonseca Lilia Geho,Ohio House votes to override governor s veto o...,The Ohio House has voted to override Gov Mike ...
1,Kiara Alfonseca Lilia Geho,Ohio Senate overrides governor veto of trans c...,The Ohio Senate voted to override Gov Mike DeW...
2,By ANDREW DeMILLO and HANNAH SCHOENBAUM AP,Efforts to Restrict Transgender Health Care En...,Republican led state legislatures are consider...
3,AMY BETH HANSON AP,Utah Joins Other States in Regulating Bathroom...,Utah became the latest state to regulate bathr...
4,Nilay Patel,Why Sen Brian Schatz thinks child safety bills...,The Democratic senator from Hawaii explains wh...
...,...,...,...
95,ABC News,British teens who killed transgender teen Bria...,A judge reveals the identities of Scarlett Jen...
96,Megan Brock,EXCLUSIVE Top Trans Pediatric Doctors Admit In...,Prominent World Professional Association for T...
97,Warner Todd Huston Warner Todd Huston,REPORT Islamic Calif High School Cancels Girls...,An Islamic high school in CA canceled a Jan gi...
98,Alana Mastrangelo Alana Mastrangelo,Transgender Democrat Permitted to Run for Ohio...,A transgender candidate has been permitted to ...
